In [1]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
driver = Edge(executable_path=r'C:\dev\WebDrivers\edgedriver_win64_111\msedgedriver.exe',options=options)

In [247]:
driver.get('https://suumo.jp/ms/chuko/fukuoka/city/')

In [248]:
search_box = driver.find_element(By.CLASS_NAME, 'searchbox')

In [250]:
search_button = search_box.find_element(By.TAG_NAME,'a').click()

In [256]:
#properties = driver.find_elements(By.XPATH, "//div[starts-with(@class,'property_unit ')]")
properties = driver.find_elements(By.XPATH, "//div[@class='property_unit-content']")

In [257]:
for prop in properties:
    price = prop.find_element(By.CLASS_NAME, 'dottable-value').text
    print(price)

250万円
580万円
590万円
598万円
698万円
850万円
898万円
898万円
949万円
998万円
998万円
998万円
1090万円
1098万円
1098万円
1180万円
1198万円
1198万円
1280万円
1280万円
1298万円
1298万円
1298万円
1298万円
1380万円
1380万円
1380万円
1398万円
1398万円
1430万円


In [219]:
prop = properties[0]
prop_info = prop.find_elements(By.CLASS_NAME, 'dottable-line')
price = prop_info[1].find_element(By.CLASS_NAME, 'dottable-value').text
print(price)
address = prop_info[2].find_element(By.TAG_NAME, 'dd').text
print(address)
closest = prop_info[2].find_elements(By.TAG_NAME, 'dd')[1].text.split('徒歩',1)[1][:-1]
print(closest)
area = prop_info[3].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0]
print(area)
ap_type = prop_info[3].find_elements(By.TAG_NAME, 'dd')[1].text
print(ap_type)
try:
    balcony = float(prop_info[4].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0])
except ValueError:
    balcony = 0
print(balcony)
date = prop_info[4].find_elements(By.TAG_NAME, 'dd')[1].text[:4]
print(date)

1900万円
福岡県福岡市東区和白丘２
8
81.91
4LDK
21.72
1994


In [106]:
index = address.find('県')
if index == -1 or index >2:
    index = address.find('府')
if index == -1 or index >2:
    index = address.find('道')
if index == -1 or index >2:
    index = address.find('都')
    tokyo=True
prefecture = address[:index]
address = address[index+1:]
index = address.find('市')
if index == -1:    
    index = address.find('区')
    special_ward = True
else:
    special_ward = False
city = address[:index]
address = address[index+1:]
if special_ward:
    ward = city
else:
    index = address.find('区')
    ward = address[:index]
    address = address[index+1:]
neighborhood = address
print(prefecture, city, ward, neighborhood)


福岡 北九州 門司 清見１


In [185]:
col_names =  ['Prefecture', 'City', 'Ward', 'Neighborhood', 'Type', 'Area m2', 'Balcony m2', 'Closest station (min)', 'Construction year', 'Price']

In [242]:
price=[]
area=[]
ap_type=[]
balcony=[]
closest=[]
year=[]
prefecture=[]
city=[]
ward=[]
neighborhood=[]

In [ ]:
while True:
    properties = driver.find_elements(By.XPATH, "//div[@class='property_unit-content']")
    price=[]
    area=[]
    ap_type=[]
    balcony=[]
    closest=[]
    year=[]
    prefecture=[]
    city=[]
    ward=[]
    neighborhood=[]
    for prop in properties:
        prop_info = prop.find_elements(By.CLASS_NAME, 'dottable-line')
        price.append(prop_info[1].find_element(By.CLASS_NAME, 'dottable-value').text)
        address = prop_info[2].find_element(By.TAG_NAME, 'dd').text
        try:
            closest.append(prop_info[2].find_elements(By.TAG_NAME, 'dd')[1].text.split('歩',1)[1][:-1])
        except IndexError:
            closest.append(60)
        area.append(prop_info[3].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0])
        ap_type.append(prop_info[3].find_elements(By.TAG_NAME, 'dd')[1].text)
        try:
            balc = float(prop_info[4].find_element(By.TAG_NAME, 'dd').text.split('m',1)[0])
        except ValueError:
            balc = 0
        balcony.append(balc)
        year.append(prop_info[4].find_elements(By.TAG_NAME, 'dd')[1].text[:4])
        index = address.find('県')
        if index == -1 or index >3:
            index = address.find('府')
        if index == -1 or index >2:
            index = address.find('道')
        if index == -1 or index >2:
            index = address.find('都')
            tokyo=True
        prefecture.append(address[:index])
        address = address[index+1:]
        index = address.find('市')
        if index == -1:    
            index = address.find('区')
            special_ward = True
        else:
            special_ward = False
        city.append(address[:index])
        address = address[index+1:]
        if special_ward:
            ward.append(city[-1])
        else:
            index = address.find('区')
            ward.append(address[:index])
            address = address[index+1:]
        neighborhood.append(address)
    try:
        properties_df = properties_df.append(pd.DataFrame(list(zip(prefecture, city, ward, neighborhood, ap_type, area, balcony, closest, year, price)), columns = col_names))
    except NameError:
        properties_df = pd.DataFrame(list(zip(prefecture, city, ward, neighborhood, ap_type, area, balcony, closest, year, price)), columns = col_names)

    pagination = driver.find_elements(By.XPATH, "//p[@class='pagination-parts']")
    if pagination[-1].text == "次へ":
        pagination[-1].find_element(By.TAG_NAME,'a').click()
    else:
        break

In [ ]:
properties_df.size

In [254]:
properties_df.shape[0]

90

In [239]:
pagination = driver.find_elements(By.XPATH, "//p[@class='pagination-parts']")

In [240]:
if pagination[-1].text == "次へ":
    pagination[-1].find_element(By.TAG_NAME,'a').click()

AttributeError: 'str' object has no attribute 'asint'

In [255]:
if 'properties_df' in locals():
    del properties_df

In [ ]:
properties_df.to_csv('bukken_data.csv')